In [ ]:
# Мои ключи к api
api_id = '28174297'
api_hash = '32bf3352feb963de902479b9a2dc3489'

# Библиотеки для парсинга
import nest_asyncio
import os
from pyrogram import Client
from tqdm.asyncio import tqdm
from telethon import TelegramClient
from datetime import datetime

In [ ]:
nestasyncio.apply()

#Создаем клиент Pyrogram: с помощью него можно получить данные о количестве комментариев и эмодзи, что в текущей версии telethon невозможно
app = Client('pyrogramsession', apiid, apihash)

#Создаем клиент Telethon: этот пакет помогает корректно считывать форматированный текст сообщений, что невозможно в pyrogram
telethonclient = TelegramClient('telethonsession', apiid, apihash)

# Создаем пустой список для хранения данных
data = []

#Определяем позитивные реакции
positivereactions = {'👍', '❤️', '🔥', '💯','❤️‍🔥', '👏'}  # Одобрительные

#Задаем диапазон дат (январь-сентябрь 2024 года)
startdate = datetime(2024, 1, 1)  # Начальная дата (1 января 2024)
enddate = datetime(2025, 9, 1)    # Конечная дата (1 сентября 2025)

#Задаем функцию для получения текста по его id с использованием telethon

async def getposttext(channelusername, postid):

    try:
        # Получаем сообщение по ID
        message = await telethonclient.getmessages(channelusername, ids=postid)
        if message:
            # Возвращаем текст или подпись (если есть)
            return message.text or getattr(message, 'caption', "") or ""
        return ""  # Если сообщение не найдено
    except Exception as e:
        print(f"Ошибка при получении сообщения {postid}: {e}")
        return ""

async def main():
    # Подключаем клиент Telethon
    await telethonclient.start()

    async with app:
        # Получаем информацию о канале по его username
        channelusernames = ['@zigmundonline', '@yasnolive', '@psyalterru']
        for channelusername in channelusernames:
            channel = await app.getchat(channelusername)

            # Получаем сообщения из канала
            messages = app.getchathistory(channel.id)

            # Обрабатываем каждое сообщение с прогресс-баром
            async for message in tqdm(messages, desc=f"Обработка сообщений в {channelusername}", unit="сообщение"):
                # Проверяем, попадает ли дата сообщения в диапазон
                if startdate <= message.date < enddate:
                    # Получаем текст сообщения
                    messagetext = message.text or getattr(message, 'caption', "") or ""

                    # Если текст отсутствует, пытаемся получить его через Telethon по ID
                    if not messagetext:
                        messagetext = await getposttext(channelusername, message.id)

                    # Получаем ID сообщения
                    messageid = message.id

                    # Получаем дату публикации
                    postdate = message.date.strftime('%Y-%m-%d %H:%M:%S')

                    # Формируем ссылку
                    postlink = f"https://t.me/{channelusername[1:]}/{messageid}"  # Убираем символ '@'

                    # Получаем количество просмотров
                    views = message.views if hasattr(message, 'views') else 0

                    # Получаем реакции
                    reactions = message.reactions if hasattr(message, 'reactions') else None

                    # Считаем положительные реакции
                    positivecount = 0
                    if reactions:
                        for reaction in reactions.reactions:  # Используем атрибут reactions
                            if hasattr(reaction, 'emoji'):
                                if reaction.emoji in positivereactions:
                                    positivecount += reaction.count

                    # Добавляем данные в список
                    data.append({
                        "postid": messageid,
                        "text": messagetext,
                        "views": views,
                        "positivereactions": positivecount,
                        "date": postdate,
                        "link": postlink,
                        "channel": channelusername
                    })
                elif message.date < startdate:
                    # Если дата сообщения раньше начала диапазона, прекращаем обработку
                    break

    # Преобразуем список в DataFrame и очищаем от пустых сообщений: если в посте несколько вложений, то
    # каждое считается за отдельную единицу сообщения, однако для анализа такие "пустые" посты непригодны
    dfdata = pd.DataFrame(data)
    dfclear = dfdata[dfdata['text'] != ""]

    # Сохраняем DataFrame в Excel-файл
    dfclear.toexcel('telegramposts2024.xlsx', index=False)
    print("Данные сохранены в файл 'telegramposts2024.xlsx'")

    # Отключаем клиент Telethon
    await telethonclient.disconnect()

    return dfclear

# Запускаем асинхронный цикл
if name == 'main':
    import asyncio
    dfclear = asyncio.run(main())

display(df_clear)